In [6]:
from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [40]:
import cadquery as cq
import sys
import os
sys.path.append('../')
from config import read_config_from_json
from plate import make_plate
from case import make_case, get_distance_between_two_shapes

config_dict = """{
    "row_key_numbers": [
        7,
        10,
        10,
        10
    ],
    "columnSpacing": 19,
    "rowSpacing": 19,
    "plateThickness": 2,
    "screwHoleDiamater": 2.4,
    "switchHoleSize": 13.9,
    "shape": 0,
    "notched_keyhole": true,
    "caseHeight": 22,
    "caseGap": 1.0,
    "wallThickness": 1.6,
    "floorThickness": 3.0,
    "edgeFillet": 3,
    "bottomFillet": 2,
    "controller": {
        "name": "Pi Pico",
        "board_dimension_x": 51.0,
        "board_dimension_y": 21.0,
        "screw_hole_x": 47.0,
        "screw_hole_y": 11.4
    },
    "controllerYOffset": 6,
    "controllerBoxThickness": 2
}"""

config = read_config_from_json(string=config_dict)

def cut_holes_in_top_plate(top):
    top_plate_width = get_distance_between_two_shapes(top.edges('|Y and >Z'))['x']
    led_num=3
    led_array_start_x = -6
    led_spacing = 7
    led_hole_dia = 5.15

    #cut holes for LEDs
    for i in range(0,led_num):
        top= (top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
            .moveTo(led_array_start_x - i*led_spacing,(-0.5))
            .circle(0.5*led_hole_dia)
            .cutThruAll()
        )

    # add slot for potentiometer
    circle_dia = 9.1
    flat_width = 8.1
    encoder_hole_centre = (16, -1)
    top = (top.faces('>Z').workplane(centerOption="CenterOfBoundBox")
        .center(encoder_hole_centre[0],encoder_hole_centre[1])
        .moveTo(-0.5*flat_width, ((0.5*circle_dia)**2 - (0.5*flat_width)**2)**0.5)
        .threePointArc((0, 0.5*circle_dia), (0.5*flat_width, ((0.5*circle_dia)**2 - (0.5*flat_width)**2)**0.5))
        .vLine(-10)
        .hLine(-flat_width)
        .vLine(10)
        .wire()
        .cutThruAll()
    )
    return top

def cut_aviator_connector_hole(case):

    aviator_hole_dia = 6.25
    aviator_flat_width = 5.3
    aviator_hole_height =  1

    case = (case.faces(">Y").workplane(centerOption='CenterOfBoundBox')
                    .center(0,aviator_hole_height)
                    .moveTo(-0.5*aviator_flat_width, -((0.5*aviator_hole_dia)**2 - (0.5*aviator_flat_width)**2)**0.5)
                    .threePointArc((0, -0.5*aviator_hole_dia), (0.5*aviator_flat_width, -((0.5*aviator_hole_dia)**2 - (0.5*aviator_flat_width)**2)**0.5))
                    .lineTo(0.5*aviator_flat_width,((0.5*aviator_hole_dia)**2 - (0.5*aviator_flat_width)**2)**0.5)
                    .threePointArc((0, 0.5*aviator_hole_dia), (-0.5*aviator_flat_width, ((0.5*aviator_hole_dia)**2 - (0.5*aviator_flat_width)**2)**0.5))
                    .close()
                    .cutBlind(until='next')
                    )
    return case 

def get_screw_positions_(config):
    print('xxxx')
    sp=[]
    x_centre = max(config.row_key_numbers)* config.rowSpacing*0.5
    row_holes = config.row_key_numbers
    nrows = len(config.row_key_numbers)
    for col_num in range(0,nrows,1):    
        row_size = row_holes[col_num]
        centre = row_size* config.rowSpacing*0.5

        if centre != x_centre:
            x_trans = x_centre - centre
        else:
            x_trans = 0
            
        if row_size%2 ==0: #for rows with even key numbers
            for row_num in [1, 0.5*row_size,row_size-1]:#range(1,row_size, 2):
                hole_x_pos = config.rowSpacing*row_num+x_trans-0.5*config.rowSpacing
                hole_y_pos = config.columnSpacing*col_num+0.0*config.columnSpacing
                sp.append((hole_x_pos, hole_y_pos))
                
        else: # for rows with odd key numbers
            for row_num in [1, row_size-1]:#range(1,row_size, 2):
                hole_x_pos = config.rowSpacing*row_num+x_trans-0.5*config.rowSpacing
                hole_y_pos = config.columnSpacing*col_num+0.0*config.columnSpacing
                sp.append((hole_x_pos, hole_y_pos))
    return sp
# case = make_case(config, 
#                 get_screw_positions=get_screw_positions_,
#                  #modify_controller_box=cut_holes_in_top_plate,
#                  #cut_hole_for_connector=cut_aviator_connector_hole
#                  )

plate= make_plate(config, get_screw_hole_positions=get_screw_positions_)
#assy = cq.Assembly()
#assy.add(case)
#assy.add(plate,loc=cq.Location(cq.Vector(0,0,18)) )

#file_location = os.path.abspath(os.path.dirname(__file__))

#assy.save( os.path.join(file_location, "KB.stl"))

xxxx


In [41]:
plate

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…